In [1]:
%config InlineBackend.figure_formats = ['svg']
import quimb.tensor as qtn
import quimb as qu
import cotengra as ctg
import autoray as ar
from tcompress import register_ as reg
from tcompress import algo_cooling as algo
from tcompress import quf
import time
import numpy as np
from quimb.tensor.belief_propagation.l2bp import L2BP
from scipy.linalg import expm
from tqdm import tqdm

In [2]:
reg.reg_complex_svd()
import torch
to_backend = algo.backend_torch(device = "cpu", dtype = torch.complex128, requires_grad=False)
to_backend_c = algo.backend_torch(device = "cpu", dtype = torch.complex128, requires_grad=False)

opt = algo.opt_(progbar=False, max_time="rate:1e9", max_repeats=128, optlib="cmaes")


In [3]:
#ITF params
J, h , delta = (1, 3.05, 0.05)

In [4]:
Lx, Ly = 3, 4
L = Lx * Ly
edges = qtn.edges_2d_square(Lx=Lx, Ly=Ly, cyclic=False)

sites = sorted({ (site,) for edge in edges for site in edge})
N = len(sites)

map_1d_2d = { Lx*j+i: (i,j)   for i in range(Lx) for j in range(Ly)}
map_2d_1d = { (i,j):Lx*j+i   for i in range(Lx) for j in range(Ly)}

inds_k = [ f"k{i}"  for i in range(L)]
inds_b = [ f"b{i}"  for i in range(L)]
ind_k_2d = []
ind_b_2d = []
for i in range(L):
        x,y = map_1d_2d[i]
        ind_k_2d.append(f"k{x},{y}")
        ind_b_2d.append(f"b{x},{y}")


In [5]:
mpo = quf.mpo_ITF_2d(Lx, Ly, data_type="float64", chi=200, cutoff_val=1.0e-12, field=h, sign="-", print_=False)
mpo

model= -* \sum<ij> Z_iZ_j + -*3.05 X


MatrixProductOperator(tensors=12, indices=35, L=12, max_bond=5)

In [6]:
Ham_ = np.array(mpo.to_dense(inds_k, inds_b), dtype="complex128")

U_matrix = expm(-1j * Ham_ * delta)
U_matrix = to_backend_c(U_matrix)
print("U shape:", U_matrix.shape, delta)
print("Unitary?", np.allclose(U_matrix.conj().T @ U_matrix, np.eye(Ham_.shape[0])))

U shape: torch.Size([4096, 4096]) 0.05
Unitary? True


In [7]:
# U_reshaped = expm(-1j * Ham_ * 1.)
# U_reshaped = to_backend_c(U_reshaped)
# print("U shape:", U_reshaped.shape)
# U_reshaped = U_reshaped.reshape([2] * 24)
# U_tn = qtn.Tensor(U_reshaped, inds=ind_k_2d+ind_b_2d ) 
# qu.save_to_disk(U_tn, "store/U_tn")

In [8]:
pepo_st = quf.pepo_trotter_ITF(edges, Lx=Lx, Ly=Ly, to_backend=to_backend_c, h=h, J=J, delta=delta)
pepo_st
pepo_ft_ = quf.pepo_trotter_ITF_fourth(edges, Lx=Lx, Ly=Ly, to_backend=to_backend_c, h=h, J=J, delta=delta)
pepo_ft_

PEPO(tensors=12, indices=41, Lx=3, Ly=4, max_bond=8)

In [9]:
pepo_st_m = pepo_st.to_dense(ind_b_2d, ind_k_2d)
pepo_ft_m = pepo_ft_.to_dense(ind_b_2d, ind_k_2d)


In [10]:
F = algo.gate_fidelity(pepo_st_m, U_matrix)
print("F-second order", F, 1- F)

F-second order 0.9999911999240306 8.800075969372934e-06


In [11]:
F = algo.gate_fidelity(pepo_ft_m, U_matrix)
print("F-fourth order", F, 1- F)

F-fourth order 0.9999999893456823 1.0654317650349299e-08


In [12]:
%%time

delta = 0.05
pepo_t = quf.pepo_trotter_ITF(edges, Lx=Lx, Ly=Ly, to_backend=to_backend_c, h=h, J=J, delta=delta)
# pepo_t = quf.pepo_trotter_ITF_fourth(edges, Lx=Lx, Ly=Ly, to_backend=to_backend_c, h=h, J=J, delta=delta)
# chi = 3
# pepo_t.compress_all(inplace=True, **{"max_bond":chi, "canonize_distance":4, "cutoff":1e-14})



chi = 12
pepo = pepo_t.copy()

Infidelity_l = []
Infidelity_bp_l = []
Norm_l = []

Error_l = []
Error_norm_bp = []
Error_norm = []
f = 1
t = delta
t_l = []
F_true = []
norm = float(2.0) ** L
for step in tqdm(range(20 - 1)):
    t += delta
    
    U_matrix = expm(-1j * Ham_ * t)
    U_matrix = to_backend_c(U_matrix)

    
    pepo = algo.apply_pepo(pepo, pepo_t, flat=True)
    pepo_raw = pepo.copy()
    #print(step, "bond", pepo.max_bond())
    
    
    bp = L2BP(pepo, site_tags=list(pepo.site_tags), damping=0.0, optimize=opt, 
              normalize = 'L2phased', update='parallel',
             )
    bp.run(tol=1.e-7, max_iterations=2000, progbar=False, diis = True)
    pepo = bp.compress(pepo, max_bond=chi, cutoff=1.-12, cutoff_mode="rsum2", renorm=0, lazy=False)
    
    mantissa, norm_exponent = bp.contract(strip_exponent=True)
    est_norm = complex(mantissa * 10**norm_exponent)
    print("bp_norm", est_norm.real, est_norm.real/2**L)
    Infidelity_bp_l.append( est_norm.real/2**L )
    # print("check",  complex((pepo_raw.H & pepo_raw).contract(all, optimize=opt)).real, complex(norm).real)
    # print("check_",  complex((pepo.H & pepo).contract(all, optimize=opt)).real)
    Error_norm_bp.append( 1 - est_norm.real/2**L  )

    
    
    chi_ = 64
    copt = algo.copt_(progbar=True, chi=chi_, directory=None, max_repeats=2**9)
    f = quf.loss_peps(pepo, pepo_raw, opt, copt, cost_f="fid", val_=norm, chi_bmps=chi_, mode="exact", progbar=False)
    #print("g")
    #f_ = quf.loss_peps_g(pepo, pepo_raw, opt, copt, cost_f="fid", chi_bmps=chi_, mode = "exact", progbar=False)

    Infidelity_l.append(1 - complex(f).real)

    norm = quf.peps_norm(pepo, opt=opt, copt=copt, chi=chi_, mode = "exact",
            max_separation=1, 
            sequence = ['xmin', 'xmax', 'ymin', 'ymax'],
            progbar=False,
            )
    Norm_l.append( complex(norm).real/(2**L)  )
    print(complex(norm).real, complex(norm).real/(2**L))
    Error_l.append( 1 - np.prod(Infidelity_l) )
    Error_norm.append( 1 - complex(norm).real/(2**L)  )

    
    print("t", t, "f", 1 - complex(f).real, np.prod(Infidelity_l), Error_l[-1])

    qu.save_to_disk(Error_norm_bp, f"store/E_compressbp_{chi}", )
    qu.save_to_disk(Error_l, f"store/E_compress_{chi}", )

    pepo_m = pepo.to_dense(ind_b_2d, ind_k_2d)
    F = algo.gate_fidelity(pepo_m, U_matrix)
    print("F_true", F)
    F_true.append(F)
    qu.save_to_disk(F_true, f"store/F_true_{chi}", )
    # print(1 - complex(f_).real)
    t_l.append(t)
    qu.save_to_disk(t_l, f"store/t_l", )
    

  0%|                                                                                                           | 0/19 [00:00<?, ?it/s]

bp_norm 4096.000000000042 1.0000000000000102
4096.000000000036 1.0000000000000089
t 0.1 f 1.0000000000000115 1.0000000000000115 -1.1546319456101628e-14


  5%|█████▏                                                                                             | 1/19 [00:08<02:24,  8.02s/it]

F_true 0.999969094718678
bp_norm 4096.000000000063 1.0000000000000153
4096.000000000104 1.0000000000000253
t 0.15000000000000002 f 1.0000000000000049 1.0000000000000164 -1.6431300764452317e-14


 11%|██████████▍                                                                                        | 2/19 [00:19<02:55, 10.32s/it]

...

F_true 0.999533204705194
bp_norm 4095.79799104967 0.9999506814086109
4100.240297908346 1.0010352289815299
t 0.9500000000000003 f 0.9999322693667007 0.9998667326822783 0.00013326731772167477


 95%|████████████████████████████████████████████████████████████████████████████████████████████▊     | 18/19 [11:40<00:57, 57.11s/it]

F_true 0.9992316474499607
bp_norm 4095.586031313081 0.9998989334260452
4100.233844070273 1.001033653337469
t 1.0000000000000002 f 0.9998762125548118 0.999742961733911 0.0002570382660890269


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [12:58<00:00, 40.95s/it]

F_true 0.9986772462048807
CPU times: user 4h 13min 17s, sys: 1h 19min 53s, total: 5h 33min 11s
Wall time: 12min 58s


In [13]:
Ham_ = np.array(mpo.to_dense(inds_b, inds_k), dtype="complex128")

print(t)
U_matrix = expm(-1j * Ham_ * t)
U_matrix = to_backend_c(U_matrix)
print("U shape:", U_matrix.shape, t)
print("Unitary?", np.allclose(U_matrix.conj().T @ U_matrix, np.eye(Ham_.shape[0])))

1.0000000000000002
U shape: torch.Size([4096, 4096]) 1.0000000000000002
Unitary? True


In [ ]:
print(delta)
pepo_t = quf.pepo_trotter_ITF_fourth(edges, Lx=Lx, Ly=Ly, to_backend=to_backend_c, h=h, J=J, delta=delta)
chi = 3
pepo_t.compress_all(inplace=True, **{"max_bond":chi, "canonize_distance":4, "cutoff":1e-14})


ind_k = []
ind_b = []
for i in range(L):
        x,y = map_1d_2d[i]
        ind_k.append(f"k{x},{y}")
        ind_b.append(f"b{x},{y}")

pepo_m = pepo.to_dense(ind_b, ind_k)
pepo_t_m = pepo_t.to_dense(ind_b, ind_k)


In [15]:
np.sqrt(np.trace(pepo_m.conj().T @ pepo_m))

(64.03306836369998+0j)

In [16]:
F = algo.gate_fidelity(pepo_m, U_matrix)
print("F=", F, 1- F)


F= 0.998677246204881 0.001322753795119036


In [17]:
F = np.prod(Infidelity_l)
F

0.999742961733911

In [18]:
1 - 0.999978789601

2.1210399000048952e-05

In [19]:
t = 0.05
U_f = pepo_t_m * 1.
F_exact = []
for step in tqdm(range(20 - 1)):
    t += 0.05
    U_f =  U_f @ pepo_t_m 
    U_matrix = expm(-1j * Ham_ * t)

    F = algo.gate_fidelity(U_f, U_matrix)
    F_exact.append(F)
    qu.save_to_disk(F_exact, f"store/F_exact", )

    print("F=", F, 1 - F)

  5%|█████▏                                                                                             | 1/19 [00:20<06:06, 20.38s/it]

F= 0.9999999407348961 5.9265103935501884e-08


 11%|██████████▍                                                                                        | 2/19 [00:39<05:34, 19.68s/it]

F= 0.9999998721664266 1.2783357339340995e-07


 16%|███████████████▋                                                                                   | 3/19 [00:59<05:16, 19.78s/it]

F= 0.9999997826635063 2.1733649369082286e-07


 21%|████████████████████▊                                                                              | 4/19 [01:14<04:29, 17.97s/it]

F= 0.9999996741881819 3.258118180804459e-07


 21%|████████████████████▊                                                                              | 4/19 [01:34<05:53, 23.55s/it]

KeyboardInterrupt



In [ ]:
F = algo.gate_fidelity(U_f, U_matrix)
print("F={F}", 1 - F)


In [ ]:
F_exact